# FFT Cupy GPU Seq-X & B715

- https://github.com/cupy

## On Sdumont18 (Sequana X)

2x Xeon Gold 6152 (22 cores), total 44 cores (88 vcores), 754 G RAM, 4x Tesla V100

Environment:

    ssh sdumont18
    module load sequana/current
    module load sdbase
    conda activate dir/env2
    conda activate --stack dir/env3
    export NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
    export NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice/

# Sequana-X

In [27]:
import numpy as np, cupy as cp, time as tm
def f() :
    t0 = -tm.time()    # <--- time measurement
    L = M = N = 576
    a = np.fromfunction( lambda i, j, k:
            np.sin ( i + j + k + 3 ), (N, M, L), dtype=cp.complex128 )
    f = cp.asarray(a)
    fft = cp.fft.fftn(f)
    s = complex(cp.sum(fft))
    t0 += tm.time()    # <--- time measurement
    print(f"S:{s*1e-5:.0f}", end='')
    print(f" | T:{t0:.4f}")

In [28]:
f()

S:270-0j | T:19.7136


In [29]:
f()

S:270-0j | T:19.4425


In [30]:
f()

S:270-0j | T:19.7182


<hr style="height:10px;border-width:0;background-color:green">

# B715

In [40]:
%%writefile gnb715.py
import numpy as np, cupy as cp, time as tm

t0 = -tm.time()    # <--- time measurement
L = M = N = 576
a = np.fromfunction( lambda i, j, k:
    np.sin ( i + j + k + 3 ), (N, M, L), dtype=cp.complex128 )
f = cp.asarray(a)
fft = cp.fft.fftn(f)
s = complex(cp.sum(fft))
t0 += tm.time()    # <--- time measurement
print(f"S:{s*1e-5:.0f}", end='')
print(f" | T:{t0:.4f}")

Overwriting gnb715.py


In [42]:
! cp gnb715.py /scratch${PWD#/prj}

In [34]:
%%writefile gnb715.srm
#!/bin/bash
#SBATCH --partition nvidia_dev # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --job-name gnb715      # Job name
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST
                                              
# Environment
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
conda activate --stack $dir/env3
cd $dir/fft
export NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
export NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice/

# Executable
EXEC="python gnb715.py"

# Start
echo '$ srun  --mpi=pmi2  -n' $SLURM_NTASKS  ${EXEC##*/}
echo '-- output -----------------------------'
srun  --mpi=pmi2  -n $SLURM_NTASKS  $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting gnb715.srm


<hr style="height:10px;border-width:0;background-color:green">

In [43]:
! sbatch gnb715.srm

Submitted batch job 1493003


In [44]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1493003  nvidia_de   R  0:00     1   24


In [1]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [2]:
! cat /scratch${PWD#/prj}/slurm-1493003.out

- Job ID: 1493003
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont3173
$ srun  --mpi=pmi2  -n 1 python gnb715.py
-- output -----------------------------
S:270-0j | T:39.5394
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


<hr style="height:10px;border-width:0;background-color:green">

In [3]:
! sbatch gnb715.srm

Submitted batch job 1493644


In [4]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1493644  nvidia_de   R  0:01     1   24


In [6]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [7]:
! cat /scratch${PWD#/prj}/slurm-1493644.out

- Job ID: 1493644
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont3131
$ srun  --mpi=pmi2  -n 1 python gnb715.py
-- output -----------------------------
S:270-0j | T:40.6159
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


<hr style="height:10px;border-width:0;background-color:green">

In [8]:
! sbatch gnb715.srm

Submitted batch job 1493703


In [9]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [ ]:
! squeue -n gnb715 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

In [10]:
! cat /scratch${PWD#/prj}/slurm-1493703.out

- Job ID: 1493703
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont3131
$ srun  --mpi=pmi2  -n 1 python gnb715.py
-- output -----------------------------
S:270-0j | T:34.3320
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


<hr style="height:10px;border-width:0;background-color:red">